In [4]:
import os
from imjoy_rpc.hypha import connect_to_server
import numpy as np
# Prepare paths for output
from tifffile import imread, imwrite
from kaibu_utils import fetch_image, features_to_mask

url = "https://github.com/bioimage-io/bioimageio-colab/releases/download/v0.1/hpa-dataset-v2-98-rgb.zip"

path2data = ""
outpath = ""
os.makedirs(outpath, exist_ok=True)

training_images = []
async def start_server(server_url,path2data, outpath):

    path2label = os.path.join(outpath, "labels")
    os.makedirs(path2label, exist_ok=True)
    
    path2source = os.path.join(outpath, "source")
    os.makedirs(path2source, exist_ok=True)
    # Connect to the server link
    server = await connect_to_server({"server_url": server_url})
    # When multiple people open the link above, they can join a common workspace as an ImJoy client
    def add_image(image, label):
        training_images.append((image, label))
        print(f"{len(training_images)} available already.")
        return
    
    def get_random_image():
        filenames = [f for f in os.listdir(path2data) if f.endswith(".tif")]
        n = np.random.randint(len(filenames)-1)
        image = imread(os.path.join(path2data, filenames[n]))
        #print(image.shape)
        new_filename = f"{len(os.listdir(path2source)) + 1}_{filenames[n]}"
        

        return image, filenames[n], new_filename,  
    
    def save_annotation(filename, newname, features, image_shape):
        mask = features_to_mask(features, image_shape)
        image = imread(os.path.join(path2data, filename))
        imwrite(os.path.join(path2source, newname), image)
        imwrite(os.path.join(path2label, newname), mask)
        
        
    await server.register_service({
        "name": "Model Trainer",
        "id": "biocolab",
        "config": {
            "visibility": "public"
        },
        "get_random_image": get_random_image,
        "save_annotation": save_annotation,
        
    })

    print("https://imjoy.io/lite?plugin=https://raw.githubusercontent.com/bioimage-io/bioimageio-colab/main/plugins/bioimageio-colab.imjoy.html")


server_url = "https://ai.imjoy.io"
await start_server(server_url, path2data, outpath)

https://imjoy.io/lite?plugin=https://raw.githubusercontent.com/bioimage-io/bioimageio-colab/main/plugins/bioimageio-colab.imjoy.html
